# Summary
We want to have the scores under the imputation scenario but without joint tratining.
- Take the code file for joint training
- Modify it to return answers before joint trainng



In [9]:
import os
import sys

import numpy as np
import pickle 


# import trainer
# -----------------------------
os.chdir('/home/david/Desktop/projects/thesis/')
sys.path.append('/home/david/Desktop/projects/thesis/')

from combined_downstream_upstream.utils.LoadModels import *
from combined_downstream_upstream.executors.train_combined_model import trainer, combined_results_without_joint_training_on_test


# upstream model params and config
# ---------------
upstream_params = {
    'input_dimension': 12,
    'output_dimension': 12,              
    'hidden_dimmension':  128,           # d_model (int) – the number of expected features in the input (required)???,
    'attention_heads': 8,               # number of attention heads, if None then d_model//64,
    'encoder_number_of_layers': 8,
    'dropout': 0.4,
    'clip': 1,
    'positional_encodings': False,
    'device':'cuda'
}

upstream_model   = load_upstream_model(upstream_params, folder_path = './upstream_seq2seq/models/', model_name = 'transformer_cnn_4heads')



Upstream Model File Path: ./upstream_seq2seq/models/transformer_cnn_4heads_1701990699.9536803.pt
Num attention heads: 8


/home/david/anaconda3/lib/python3.9/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [10]:
results = {'turned_off_channels':[], 'roc_auc':[], 'pr_auc':[]}

for channels in range(12):
    try:
        downstream_models_root = f"./downstream_classification/models/AF-V10-different-seeds-ptb-{channels}_channels_off-saving-downstream-models"
        downstream_models_names = [x for x in os.listdir(downstream_models_root) if x.endswith('.pt')]

        for model_path in downstream_models_names:
            downstream_model = load_downstream_model(
                dropout=0.5,scale=1,num_inputs=12, 
                weights_path=f"{downstream_models_root}/{model_path}"
                )

            aucpr, rocauc = combined_results_without_joint_training_on_test(
                metadata_file_path      = './downstream_classification/data/combined_data/ptb_signal_level_metadata_with_label.csv',
                data_folder_path        = './downstream_classification/data/ptb-ecg-processed-divided-into-450/', 
                targets                 = ['AF'],
                batch_size              = 128, 
                impute_only_missing     = True,
                upstream_model          = upstream_model,
                downstream_model        = downstream_model,
                continue_training_upstream_model = False,
                leads = ['LI', 'LII', 'LIII', 'aVL', 'aVR','aVF', 'V1','V2','V3','V4','V5','V6'],
                loss_function_weight = None,

                reconstruction_loss_weight = 0,

                internal_data=False,
                channels_to_turn_off = channels,

                )
            
            results['turned_off_channels'].append(channels)
            results['roc_auc'].append(rocauc)
            results['pr_auc'].append(aucpr)
    except:
        continue

import pandas as pd
results_df = pd.DataFrame(results)
results_df.to_csv("/home/david/Desktop/projects/thesis/downstream_classification/notebooks/imputation_withought_joint_training.csv", index=False)

Within epoch loss (validation) 0.34061: 100%|██████████| 109/109 [00:27<00:00,  3.90it/s]


In [12]:
results_df

,turned_off_channels,roc_auc,pr_auc
0,0,0.985775,0.891715
1,0,0.985572,0.892803
2,0,0.983436,0.900288
3,0,0.985136,0.897007
4,0,0.986414,0.901569
...,...,...,...
134,11,0.482159,0.095167
135,11,0.633818,0.165297
136,11,0.540634,0.135952
137,11,0.516674,0.103052


In [24]:
roc_auc_se   = pd.DataFrame((results_df.groupby(['turned_off_channels'])['roc_auc'].sem() * 100).round(2)).reset_index()
roc_auc_mean = pd.DataFrame((results_df.groupby(['turned_off_channels'])['roc_auc'].mean() * 100).round(2)).reset_index()
pr_auc_se    = pd.DataFrame((results_df.groupby(['turned_off_channels'])['pr_auc'].sem() * 100).round(2)).reset_index()
pr_auc_mean  = pd.DataFrame((results_df.groupby(['turned_off_channels'])['pr_auc'].mean() * 100).round(2)).reset_index()

roc_auc_df = roc_auc_mean.merge(roc_auc_se, on = 'turned_off_channels')
roc_auc_df['Resulls'] = [f'{x} ({y})' for x,y in zip(roc_auc_df['roc_auc_x'],roc_auc_df['roc_auc_y'])]

pr_auc_df = pr_auc_mean.merge(pr_auc_se, on = 'turned_off_channels')
pr_auc_df['Resulls'] = [f'{x} ({y})' for x,y in zip(pr_auc_df['pr_auc_x'],pr_auc_df['pr_auc_y'])]


display(roc_auc_df,pr_auc_df)

,turned_off_channels,roc_auc_x,roc_auc_y,Resulls
0,0,98.43,0.04,98.43 (0.04)
1,1,98.16,0.04,98.16 (0.04)
2,2,97.88,0.05,97.88 (0.05)
3,3,97.43,0.12,97.43 (0.12)
4,4,96.30,0.54,96.3 (0.54)
5,5,95.19,0.46,95.19 (0.46)
6,6,95.05,0.43,95.05 (0.43)
7,7,91.11,1.62,91.11 (1.62)
8,8,89.65,1.78,89.65 (1.78)
9,9,85.80,1.61,85.8 (1.61)


,turned_off_channels,pr_auc_x,pr_auc_y,Resulls
0,0,89.31,0.17,89.31 (0.17)
1,1,85.86,0.36,85.86 (0.36)
2,2,83.19,0.30,83.19 (0.3)
3,3,78.96,0.70,78.96 (0.7)
4,4,72.48,2.01,72.48 (2.01)
5,5,68.62,1.63,68.62 (1.63)
6,6,66.09,1.56,66.09 (1.56)
7,7,53.94,3.59,53.94 (3.59)
8,8,48.36,3.85,48.36 (3.85)
9,9,38.07,2.61,38.07 (2.61)


In [11]:
results_df.groupby(['turned_off_channels'])['roc_auc'].mean(),results_df.groupby(['turned_off_channels'])['pr_auc'].mean()

(turned_off_channels
 0     0.984305
 1     0.981555
 2     0.978849
 3     0.974274
 4     0.963028
 5     0.951856
 6     0.950464
 7     0.911143
 8     0.896548
 9     0.858006
 10    0.752559
 11    0.575036
 Name: roc_auc, dtype: float64,
 turned_off_channels
 0     0.893123
 1     0.858587
 2     0.831859
 3     0.789625
 4     0.724760
 5     0.686179
 6     0.660884
 7     0.539437
 8     0.483562
 9     0.380656
 10    0.249087
 11    0.133648
 Name: pr_auc, dtype: float64)